The code is built based on the supplementary files in [1].

[1] Tong, X., Xu, X., Huang, S. L., & Zheng, L. (2021). A Mathematical Framework for Quantifying Transferability in Multi-source Transfer Learning. Advances in Neural Information Processing Systems, 34

1. Load the packages

In [ ]:

import scipy.io
import random
import numpy as np
import torch
import torch.nn as nn
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import torch
import torchvision
import torch.nn as nn
import time
from torchvision import models
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot
import torch
import torch.nn as nn
from torch.autograd.function import Function
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from  torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import random
from torch.autograd import Variable
import torch.nn.functional as F


test_cuda=True
device = torch.device("cuda" if test_cuda else "cpu")
time_start=time.time()

2. Load Office-Caltech-10 dataset


You need to download [Office-Caltech-10 DeCAF-fc6 features](https://drive.google.com/drive/folders/1XAKj5ikc6ygcanexeiWuwvnHMf5rUVGy) generated in [1] to run this code. Then: 


2.1. If you run the code on Colab, you will need to put the data in the corresponding folder of your [Google Drive](https://drive.google.com/drive/u/0/my-drive).


2.2. If you run the code locally, you will need to put the data in the corresponding folder of your device.



[1] Donahue, J., Jia, Y., Vinyals, O., Hoffman, J., Zhang, N., Tzeng, E., & Darrell, T. (2014, January). Decaf: A deep convolutional activation feature for generic visual recognition. In International conference on machine learning (pp. 647-655). PMLR.

In [ ]:
# Mount the Google drive, please ignore this cell if you run the code locally.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
amazon = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/amazon_fc6.mat') # The path where you put the data.
webcam = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/webcam_fc6.mat') # The path where you put the data.
dslr = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/dslr_fc6.mat')     # The path where you put the data.

3. Select 10 classess data on office31 dataset

In [ ]:

amazon_fts,amazon_labels=amazon['fts'],amazon['labels'].reshape(-1)-1
webcam_fts,webcam_labels=webcam['fts'],webcam['labels'].reshape(-1)-1
dslr_fts,dslr_labels=dslr['fts'],dslr['labels'].reshape(-1)-1
print(amazon_fts.shape)
print(amazon_labels)
print(webcam_labels.shape)
print(dslr_labels.shape)
n_class=10

np.random.choice
class_set=np.array([1,2,6,11,12,13,16,17,18,23])-1
#backpack, bike, calculator, headphones, keyboard, laptop-computer, monitor, mouse, mu, and projector

def get_data(domain_name):
  index=np.array([])
  label_for_training=[]
  fts,labels=None, None
  if domain_name=='amazon':
    fts,labels=amazon_fts,amazon_labels
  elif domain_name=='webcam':
    fts,labels=webcam_fts,webcam_labels
  elif domain_name=='dslr':
    fts,labels=dslr_fts,dslr_labels

  j=0
  for i in class_set:
    index=np.hstack((index,np.where(labels==i)[0]))
    for a in np.where(labels==i)[0]:
      label_for_training.append(j)
    j=j+1
  index=index.astype(int)
  return fts[index],np.array(label_for_training)



(2817, 4096)
[ 0  0  0 ... 30 30 30]
(795,)
(498,)


4. Example on *Amaon and DSLR* domain adaptation task.

In [ ]:
x1,y1=get_data('amazon')
x2,y2=get_data('dslr')

5. Mathematical Framework (MF method)

In [ ]:

index=np.random.choice(x2[y2==0].shape[0],3,replace=False)
refx=x2[y2==0][index]
refy=[0,0,0]
for i in range(1,10):
    index=np.random.choice(x2[y2==i].shape[0],3,replace=False)
    refx=np.vstack((refx,x2[y2==i][index]))
    refy=np.append(refy,[i,i,i])
refy=refy.astype(int)

index=np.random.choice(x1[y1==0].shape[0],8,replace=False)
sourcex=x1[y1==0][index]
sourcey=0*np.ones(8)
for i in range(1,10):
    index=np.random.choice(x1[y1==i].shape[0],8,replace=False)
    sourcex=np.vstack((sourcex,x1[y1==i][index]))
    sourcey=np.append(sourcey,i*np.ones(8))
sourcey=sourcey.astype(int)


#feature---------------
class Net_f(nn.Module):
    def __init__(self):
        super(Net_f, self).__init__()
        self.fc1 = nn.Linear(4096,1024)
        self.act1= nn.PReLU()

        self.fc2 = nn.Linear(1024,64)
        self.act2= nn.PReLU()

    def forward(self,x):
        out=F.relu(self.fc1(x))
        out=self.act1(out)
        out=self.fc2(out)
        out=self.act2(out)
        return out       


class Net_g(nn.Module):
    def __init__(self,num_class=10, dim=64):
        super(Net_g, self).__init__()

        self.fc=nn.Linear(num_class, dim)

    def forward(self,x):
        out=self.fc(x)

        return out

def corr(f,g):
    k = torch.mean(torch.sum(f*g,1))
    return k
    
def cov_trace(f,g):
    cov_f = torch.mm(torch.t(f),f) / (f.size()[0]-1.)
    cov_g = torch.mm(torch.t(g),g) / (g.size()[0]-1.)
    return torch.trace(torch.mm(cov_f, cov_g))

def neg_hscore(f,g):
    f0 = f - torch.mean(f,0)
    g0 = g - torch.mean(g,0)
    corr = torch.mean(torch.sum(f0*g0,1))
    cov_f = torch.mm(torch.t(f0),f0) / (f0.size()[0]-1.)
    cov_g = torch.mm(torch.t(g0),g0) / (g0.size()[0]-1.)
    return - corr + torch.trace(torch.mm(cov_f, cov_g)) / 2.

lr=0.0002
epoch=100
ind=0
model_f = Net_f().to(device)
model_g = Net_g().to(device)
optimizer_fg = torch.optim.Adam(list(model_f.parameters())+list(model_g.parameters()),lr=lr)
losslist=[]
acclist=[0]
alpha=[0.9,0.1]

samples_ref=torch.from_numpy(refx)
labels_ref=torch.from_numpy(refy)
labels_one_hot_ref = torch.zeros(len(labels_ref), 10).scatter_(1, labels_ref.view(-1,1), 1)
samples_trans=torch.from_numpy(sourcex)
labels_trans=torch.from_numpy(sourcey)
labels_one_hot_trans= torch.zeros(len(labels_trans), 10).scatter_(1, labels_trans.view(-1,1), 1)

for i in range(epoch):
    model_f.train()
    model_g.train()
    
    f_ref=model_f(Variable(samples_ref).float().to(device))
    g_ref=model_g(Variable(labels_one_hot_ref).float().to(device))
    f0_ref = f_ref - torch.mean(f_ref,0)
    g0_ref = g_ref - torch.mean(g_ref,0)
    f_trans=model_f(Variable(samples_trans).float().to(device))-torch.mean(f_ref,0)
    g_trans=model_g(Variable(labels_one_hot_trans).float().to(device))- torch.mean(g_ref,0)
    optimizer_fg.zero_grad()
    
    loss=(-2)*alpha[0]*corr(f0_ref,g0_ref)
    loss+=(-2)*alpha[1]*corr(f_trans,g_trans)
    loss+=cov_trace(f0_ref,g0_ref)
    losslist.append(loss.item())
    loss.backward()
    optimizer_fg.step()
    ind+=1
#------acc
    model_f.eval()
    model_g.eval()
    fc = model_f(Variable(samples_trans).float().to(device)).data.cpu().numpy()
    f_mean = np.sum(fc,axis=0)/fc.shape[0]
    labellist = torch.Tensor(np.eye(10))
    gc = model_g(Variable(labellist).to(device)).data.cpu().numpy()
    gce = np.sum(gc,axis=0)/gc.shape[0]
    gcp = gc-gce

    samples_test=torch.from_numpy(x2)
    labels_test = y2
    fc=model_f(Variable(samples_test).float().to(device)).data.cpu().numpy()
    fcp=fc-f_mean
    fgp=np.dot(fcp,gcp.T)
    acc = (np.argmax(fgp, axis = 1) == labels_test).sum()
    total = len(samples_test)

    samples_test=torch.from_numpy(refx)
    labels_test = refy
    fc=model_f(Variable(samples_test).float().to(device)).data.cpu().numpy()
    fcp=fc-f_mean
    fgp=np.dot(fcp,gcp.T)
    acc1 = (np.argmax(fgp, axis = 1) == labels_test).sum()
    total1 = len(samples_test)

    acc=(acc-acc1)/(total-total1)
#    print(acc)
    if acc > 0.5:
       if acc > (max(acclist)):
           paraf=model_f.state_dict()
           parag=model_g.state_dict()
#           print('changepara')
           finalacc=acc
    acclist.append(acc)
print(total-total1)
print("Best testing accuracy among epochs:",finalacc)
print("Accuracy list for epochs:",acclist)

model_fa = Net_f().to(device)
model_fa.load_state_dict(paraf)

fstar_ref=model_f(Variable(samples_ref).float().to(device))
ftilde_ref = fstar_ref - torch.mean(fstar_ref,0)
py_ref=np.zeros(31)
for i in range(31):
    py_ref[i]=np.sum(labels_ref.numpy()==i)/labels_ref.shape[0]


fstar_trans=model_f(Variable(samples_trans).float().to(device))
ftilde_trans = fstar_trans - torch.mean(fstar_ref,0)
py_trans=np.zeros(10)
for i in range(10):
    py_trans[i]=np.sum(labels_trans.numpy()==i)/labels_trans.shape[0]
lambdaf= torch.mm(torch.t(ftilde_ref),ftilde_ref) / (ftilde_ref.size()[0]-1.)

v_ref=0
for i in range(10):
    a=ftilde_ref[labels_ref==i]
    v_ref+=(1/labels_ref.shape[0]*torch.trace(torch.mm(torch.inverse(lambdaf),(torch.mm(torch.t(a),a)/a.size()[0])))).item()
    v_ref-=(1/labels_ref.shape[0]*py_ref[i]*(torch.mm(torch.mean(a,0).reshape(1,64),torch.inverse(lambdaf)).mm(torch.t(torch.mean(a,0).reshape(1,64))))).item()
v_trans=0
for i in range(10):
    a=ftilde_trans[labels_trans==i]
    v_trans+=(1/labels_trans.shape[0]*py_trans[i]/py_ref[i]*torch.trace(torch.mm(torch.inverse(lambdaf),(torch.mm(torch.t(a),a)/a.size()[0])))).item()
    v_trans-=(1/labels_trans.shape[0]*py_trans[i]*py_trans[i]/py_ref[i]*(torch.mm(torch.mean(a,0).reshape(1,64),torch.inverse(lambdaf)).mm(torch.t(torch.mean(a,0).reshape(1,64))))).item()

h_ref=torch.zeros((10,64))
for i in range(10):
    a=ftilde_ref[labels_ref==i]
    h_ref[i]=py_ref[i]*torch.mean(a,0)

h_trans=torch.zeros((10,64))
for i in range(10):
    a=ftilde_trans[labels_trans==i]
    h_trans[i]=py_trans[i]*torch.mean(a,0)

h=torch.zeros((64,64))
for i in range(10):
    h+=1/py_ref[i]*torch.mm(torch.t(h_ref[i]-h_trans[i]).reshape(64,1),(h_ref[i]-h_trans[i]).reshape(1,64))

d=torch.trace(torch.mm(torch.inverse(lambdaf),h.to(device)))

alpha=v_ref/(v_ref+v_trans+d.item())
print("Alpha:",alpha)
time_end=time.time()


127
Best testing accuracy among epochs: 0.8740157480314961
Accuracy list for epochs: [0, 0.6062992125984252, 0.6929133858267716, 0.6692913385826772, 0.6062992125984252, 0.6141732283464567, 0.7007874015748031, 0.7795275590551181, 0.7795275590551181, 0.8110236220472441, 0.84251968503937, 0.8267716535433071, 0.84251968503937, 0.84251968503937, 0.8503937007874016, 0.8661417322834646, 0.8740157480314961, 0.8661417322834646, 0.8740157480314961, 0.8503937007874016, 0.8346456692913385, 0.8346456692913385, 0.8267716535433071, 0.8346456692913385, 0.8110236220472441, 0.8110236220472441, 0.7874015748031497, 0.7874015748031497, 0.7559055118110236, 0.7637795275590551, 0.7322834645669292, 0.7165354330708661, 0.7007874015748031, 0.6771653543307087, 0.6535433070866141, 0.6456692913385826, 0.6456692913385826, 0.6377952755905512, 0.6377952755905512, 0.6220472440944882, 0.6220472440944882, 0.6220472440944882, 0.6141732283464567, 0.6062992125984252, 0.5905511811023622, 0.5905511811023622, 0.582677165354330